Feature extraction from pack P180_2,3,4 and P190_2,3,4

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import pandas as pd
import glob
import numpy as np
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
from tables.table import Column
import seaborn as sns
import scipy.stats
import plotly.express as px
from IPython.display import clear_output
from scipy.interpolate import interp1d
from datetime import datetime

In [ ]:
#set working directory containg the Data
os.chdir('/content/drive/Shareddrives/R&D/Projects/DataforManuscript/')

In [ ]:
#!pip install plotly

# Feature extraction function

In [ ]:

def Nyquist_feature_extract(Zreal,Zimg,num_interp,plotflag,Zreal_exclude=[],num_nearest_smooth=10,portion_from_end=20,turn_correction=0,defined_index_start=3,defined_index_end=-5,defined_turn=0):
#define the fucntion to interpolate nyquist plot, find features, plot and return the feature valuesinput filename, number of data points for interpolation. Input Zreal, Zimg are numpy arrays
#to return features: ymax=maximum of Zimg, xofymax=zreal value of maximum of Zimg,intercept=intercept with x-axis,diameter=diameter of semicircle, tailhead=start of linear tail,
#slope=slope of linear tail,shoulder=the beginning of second semicircle or zero
#shape=representation of shape (center-0.5max)/(center-0.75max),shoulder=min(2nd dervative before the tip of the large circle)

#Zreal and Zimg are input of real and imaginary inputs in dataframe format
#num_interp is the total number of points to interpolate the raw data onto
#plotflag is the flag to plot interactive plot using plotly
#Zreal_exclude is the Zreal value of datapoint to exlucde, e.g. [0.0005,0.0006]
#num_nearest_smooth is the number of neignboring data point used to smooth the curve for finding the start of tail
#portion_from_end is used to discard the detected start of tail if it is too close to the end of data
#turn_correction is the number of interpolated data points to correct from detected turn
#defined_index_start and defined_index_end are used for detectbing shoulder in the case of two semicircules. They can be set to exclude the nosiy low Zimg region and the flat region
#defined_turn is the option to set Zreal value of turn



  #data = pd.read_csv(filename, delimiter = ',', header = 0, encoding='latin1')
  toprint=" " #filename.split("/")

  #convert from dataframe to array
  x=Zreal
  y=Zimg
  x=x[y>-0.001]
  y=y[y>-0.001]

  #delete datapoints from Zreal_exclude
  for exclude in Zreal_exclude:
    x = np.delete( x,np.abs(x - exclude).argmin())
    y = np.delete( y,np.abs(x - exclude).argmin())


  #interpolate Zreal and Zimg above-x-axis part
  f = interp1d(x, y)
  #f2 = interp1d(x, y, kind='cubic')
  xnew = np.linspace(np.min(x), np.max(x), num=num_interp, endpoint=True)
  ynew=f(xnew)

  ################################find features#############################################################
  #initialize outputs in case there are errors
  xofymax=0
  ymax=0
  tailhead=0
  intercept=0
  slope=0
  diameter=0
  shape=0
  shoulder=0

  x_tail=[1] * 9
  y_tail=[1] * 9
  x_fit=[1]*9
  coef = np.polyfit(x_tail,y_tail,1)
  poly1d_fn = np.poly1d(coef)

  center_HM=0
  leftcross_HM=0
  rightcross_HM=0
  center_QM=0
  leftcross_QM=0
  rightcross_QM=0


  #find intercept, rough maximum and separate data to before and after max

  positive=np.where(ynew>0) #find the positive part

  #find the top of the semicircle

  def running_mean_cumsum(x, N):
    #average mean filter with sum of nearby points
    cumsum = np.cumsum(np.insert(x, 0, 0))
    return (cumsum[N:] - cumsum[:-N]) / float(N)

  #apply average filter on y and propogate x data points to find roughly where the turn is
  y_convolved = running_mean_cumsum(y[defined_index_start:defined_index_end], num_nearest_smooth)
  x_convolved = np.linspace(np.min(x[defined_index_start:defined_index_end]), np.max(x[defined_index_start:defined_index_end]), num=len(y_convolved), endpoint=True)

  turn_convolved=np.where(np.gradient(np.gradient(y_convolved, 0.0001), 0.0001)==max(np.gradient(np.gradient(y_convolved, 0.0001), 0.0001)))

  #find the tip of semicircle
  if defined_turn > 0:
    turn_initial=np.abs(xnew - defined_turn).argmin()
    tip=np.where(ynew==np.max(ynew[0:turn_initial-turn_correction]))
    start=turn_initial
  elif np.abs(np.where(ynew==np.max(ynew))[0][0]-num_interp) < round(num_interp/portion_from_end):
    # if the tailend is higher than semicicle
    turn_initial=np.abs(xnew - x_convolved[turn_convolved]).argmin()
    tip=np.where(ynew==np.max(ynew[0:turn_initial+turn_correction]))
  else:
    # if the tailend is lower than semicircle
    tip=np.where(ynew==np.max(ynew))

  #split data to two parts, one beofre the top of semicirlce, the other after
  beforetip=ynew[0:tip[0][0]]
  aftertip=ynew[tip[0][0]:-1]
  if defined_turn==0:
    turn=np.where(aftertip==np.min(aftertip)) #find the start of tail
    start=turn[0][0]

  try:
    # find half max and 0.75 max lines
    #find the region before the tail and after the tip of semicircle
    rightcircle=aftertip[0:start]
    #find values at half maximum of top of the circle
    rightcross_HM=xnew.flat[np.abs(rightcircle - ynew[tip[0][0]]/2).argmin()+tip[0][0]]
    #find values at 0.75 maximum of top of the circle
    rightcross_QM=xnew.flat[np.abs(rightcircle - 0.75*ynew[tip[0][0]]).argmin()+tip[0][0]]


    #find the leftcross and center for halfmax or 0.75 max of the semicircle
    leftcross_HM=xnew.flat[np.abs(beforetip - np.max(beforetip)/2).argmin()]
    center_HM=(leftcross_HM+rightcross_HM)/2
    leftcross_QM=xnew.flat[np.abs(beforetip - 0.75*np.max(beforetip)).argmin()]
    center_QM=(leftcross_QM+rightcross_QM)/2

    try:
      #find the shoulder, beginning of second semicircle by minimum 1st deriative
      beforetip_y=ynew[(ynew>0) & (xnew<leftcross_QM)] #[positive[0][0]:tip[0][0]-50]
      beforetip_x=xnew[(ynew>0) & (xnew<leftcross_QM)] #[positive[0][0]:tip[0][0]-50]
      beforetip_y_convolved = running_mean_cumsum(beforetip_y, 3)
      beforetip_x_convolved = np.linspace(np.min(beforetip_x), np.max(beforetip_x), num=len(beforetip_y_convolved), endpoint=True)
      shoulder_convolved=np.where(np.gradient(beforetip_y_convolved, 0.0001)==min(np.gradient(beforetip_y_convolved, 0.0001)))
      shoulder=beforetip_x_convolved[shoulder_convolved[0][0]]
      if abs(shoulder-leftcross_QM)<0.0006:
        shoulder=0
    except:
      print("Error in Nyquist_feature_extract(): shoulder is wrong")


    try:
      #extract slope for the tail with linear fit
      if defined_turn>0:
        x_tail=xnew[start:-1]
        y_tail=ynew[start:-1]
      else:
        x_tail=xnew[len(xnew)-round(0.7*(len(xnew)-start-tip[0][0])):-1]
        y_tail=ynew[len(xnew)-round(0.7*(len(xnew)-start-tip[0][0])):-1]

      x_fit=np.linspace(0,0.2,100)
      coef = np.polyfit(x_tail,y_tail,1)
      poly1d_fn = np.poly1d(coef)


      #assign values to turn
      intercept=xnew[positive[0][0]]
      xofymax=xnew[tip[0][0]]
      ymax=ynew[tip[0][0]]
      if defined_turn==0:
        tailhead=xnew[start+tip[0][0]]
      else:
        tailhead=xnew[start]
      slope=coef[0]
      diameter=2*(center_HM-xnew[positive[0][0]])
      shape=(center_HM-leftcross_HM)/(center_QM-leftcross_QM)#(leftcross_QM-intercept)/(leftcross_HM-intercept)
    except:
      print("Error in Nyquist_feature_extract(): linear region empty")
  except:
    print("Error in Nyquist_feature_extract(): start of tail is wrong")


  ################################################plot features#########################################################################
  if plotflag==1:
    #plot raw and interpoplated data
    fig = px.line(x=xnew, y=ynew,color_discrete_sequence=["black"],title=toprint[-1], width=800,height=400)
    fig.add_scatter( x=Zreal, y=Zimg,mode="markers",name="raw data")

    #plot fitted line for tail
    fig.add_scatter( x=x_tail, y=y_tail,mode="markers",name="diffusion line")
    fig.add_scatter( x=x_fit, y=poly1d_fn(x_fit),mode="lines",name="diffusion line fitting")

    #plot vertical lines for intercept, maximum, and start of tail
    fig.add_vline(x = xofymax, line_width=2, line_color="green", \
            label=dict(
            text="maximum",
            textposition="end",
            font=dict(size=12, color="green"))) #start of tail
    fig.add_vline(x = tailhead, line_width=2, line_color="black", \
            label=dict(
            text="start of tail",
            textposition="end",
            font=dict(size=12, color="black"))) #start of tail
    fig.add_vline(x = intercept, line_width=2, line_color="red", \
            label=dict(
            text="intercept",
            textposition="end",
            font=dict(size=12, color="red"))) #x-axis intercept

    #plot lines to find center by half max
    fig.add_vline(x = center_HM, line_width=2, line_color="orange", \
            label=dict(
            text="",
            textposition="end",
            font=dict(size=12, color="orange"))) #center of semicircle
    fig.add_vline(x = leftcross_HM, line_width=2, line_color="orange", \
            label=dict(
            text="",
            textposition="end",
            font=dict(size=12, color="orange"))) #half max
    fig.add_vline(x = rightcross_HM, line_width=2, line_color="orange", \
            label=dict(
            text="",
            textposition="end",
            font=dict(size=12, color="orange"))) #half max
    fig.add_hline(y = ynew[tip[0][0]]/2, line_width=2, line_color="orange", \
            label=dict(
            text="half max",
            textposition="end",
            font=dict(size=12, color="orange"))) #half max

    #plot lines to find center by quarter max
    fig.add_vline(x = center_QM, line_width=2, line_color="purple", \
            label=dict(
            text="",
            textposition="end",
            font=dict(size=12, color="purple"))) #center of semicircle
    fig.add_vline(x = leftcross_QM, line_width=2, line_color="purple", \
            label=dict(
            text="",
            textposition="end",
            font=dict(size=12, color="purple"))) #three quarters max
    fig.add_vline(x = rightcross_QM, line_width=2, line_color="purple", \
            label=dict(
            text="",
            textposition="end",
            font=dict(size=12, color="purple"))) #three quarters max
    fig.add_hline(y = 0.75*ynew[tip[0][0]], line_width=2, line_color="purple", \
            label=dict(
            text="0.75 max",
            textposition="end",
            font=dict(size=12, color="purple"))) #half max

    #plot line at the beginning of the second circle
    fig.add_vline(x = shoulder, line_width=2, line_color="blue", \
            label=dict(
            text="shoulder",
            textposition="end",
            font=dict(size=12, color="blue"))) #center of semicircle


    fig.update_yaxes(range=[0, max(y)])
    fig.update_xaxes(range=[0.9*min(x), max(x)])
    fig.show()
  return xofymax,intercept,tailhead,slope,diameter,shape,shoulder,ymax

#xofymax,intercept,tailhead,slope,diameter,shape,shoulder,ymax=Nyquist_feature_extract(x,y,500,0,defined_turn=0.045) #filename, number of datapoints to interpolate, flag 1 to plot

# Extraction features from packs

In [ ]:
# @title
#extract features in pack P180-02,3,4
my_dict = {}
for i in [2,3,4]:
  #get filenames of EIS from all cells
  directory = r'Data/P180/P180_0'+str(i)+'/'
  all_files = glob.glob(os.path.join(directory,  r'EIS_*/2_EIS_*_P180_0'+str(i)+'_*.csv'))
  n=len(all_files)

  #initialize dataframes for features for each pack
  #features: ymax=maximum of Zimg, intercept=intercept with x-axis,diameter=diameter of semicircle, tailhead=start of linear tail, slope=slope of linear tail, shape=representation of shape (center-0.5max)/(center-0.75max)
  for x in [r"pack_xofymax_P180_0"+str(i),r"pack_intercept_P180_0"+str(i),r"pack_diameter_P180_0"+str(i),r"pack_tailhead_P180_0"+str(i),r"pack_slope_P180_0"+str(i),r"pack_shape_P180_0"+str(i),\
            r"pack_shoulder_P180_0"+str(i),r"pack_ymax_P180_0"+str(i)]:
    my_dict[x] =pd.DataFrame(columns=["scan"+str(j) for j in [2,4,6,8]],index=["P180_0"+str(i)+"_"+str(j) for j in range(1,6)])


  for filename in all_files:
    data = pd.read_csv(filename, delimiter = ',', header = 0, encoding='latin1')

    pre=filename.split("/")
    key=pre[-1].split("_")
    cell=key[5].split(" ")

    x=data["Z' (Ohms)"].to_numpy()
    y=data['-Z" (Ohms)'].to_numpy()

    print(["scan"+key[2] + "_P180_" + key[4]+"_"+cell[0]])

    xofymax,intercept,tailhead,slope,diameter,shape,shoulder,ymax=Nyquist_feature_extract(x,y,500,0)

    my_dict[r"pack_xofymax_P180_0"+str(i)]["scan"+str(int(key[2]))]["P180_0"+str(i)+"_"+cell[0]]=xofymax #x of maximum of Zimg
    my_dict[r"pack_intercept_P180_0"+str(i)]["scan"+str(int(key[2]))]["P180_0"+str(i)+"_"+cell[0]]=intercept #intercept
    my_dict[r"pack_tailhead_P180_0"+str(i)]["scan"+str(int(key[2]))]["P180_0"+str(i)+"_"+cell[0]]=tailhead #start of tail
    my_dict[r"pack_slope_P180_0"+str(i)]["scan"+str(int(key[2]))]["P180_0"+str(i)+"_"+cell[0]]=slope #slope of tail
    my_dict[r"pack_diameter_P180_0"+str(i)]["scan"+str(int(key[2]))]["P180_0"+str(i)+"_"+cell[0]]=diameter #diameter of semicircle obtained from half max
    my_dict[r"pack_shape_P180_0"+str(i)]["scan"+str(int(key[2]))]["P180_0"+str(i)+"_"+cell[0]]=shape #shape=representation of shape (0.75max-Rs)/(0.5max-Rs)
    my_dict[r"pack_shoulder_P180_0"+str(i)]["scan"+str(int(key[2]))]["P180_0"+str(i)+"_"+cell[0]]=shoulder #
    my_dict[r"pack_ymax_P180_0"+str(i)]["scan"+str(int(key[2]))]["P180_0"+str(i)+"_"+cell[0]]=ymax #maximum of Zimg

  #stack features of one pack together to save
  Feature=pd.concat([my_dict["pack_"+Feature_name+"_P180_0"+str(i)] for Feature_name in ["xofymax","intercept","diameter","tailhead","slope","shape","shoulder","ymax"]])
  currentDateTime = datetime.now().strftime("%m-%d-%Y %H-%M-%S %p")
  Feature.to_csv("Generated Features/P180_0"+str(i)+f"_features {currentDateTime}.csv", index = True)

In [ ]:
# @title
#extract features in pack P190_02
my_dict = {}
for i in [2]:
  #get filenames of EIS from all cells
  directory = r'Data/P190/P190_0'+str(i)+'/'
  all_files = glob.glob(os.path.join(directory,  r'EIS_*/2_EIS_*_P190_0'+str(i)+'_*'))
  n=len(all_files)

  #initialize dataframes for features for each pack
  for x in [r"pack_xofymax_P190_0"+str(i),r"pack_intercept_P190_0"+str(i),r"pack_diameter_P190_0"+str(i),r"pack_tailhead_P190_0"+str(i),r"pack_slope_P190_0"+str(i),r"pack_shape_P190_0"+str(i),r"pack_shoulder_P190_0"+str(i),r"pack_ymax_P190_0"+str(i)]:
    my_dict[x] =pd.DataFrame(columns=["scan"+str(j) for j in [2,4,6,8]],index=["P190_0"+str(i)+"_"+str(j) for j in range(1,6)])


  for filename in all_files:
    data = pd.read_csv(filename, delimiter = ',', header = 0, encoding='latin1')

    pre=filename.split("/")
    key=pre[-1].split("_")
    cell=key[5].split(" ")

    x=data["Z' (Ohms)"].to_numpy()
    y=data['-Z" (Ohms)'].to_numpy()

    #print(["scan"+key[2] + "_P190_" + key[4]+"_"+cell[0]])
    xofymax,intercept,tailhead,slope,diameter,shape,shoulder,ymax=Nyquist_feature_extract(x,y,500,0)

    my_dict[r"pack_xofymax_P190_0"+str(i)]["scan"+str(int(key[2]))]["P190_0"+str(i)+"_"+cell[0]]=xofymax #x of maximum of Zimg
    my_dict[r"pack_intercept_P190_0"+str(i)]["scan"+str(int(key[2]))]["P190_0"+str(i)+"_"+cell[0]]=intercept #intercept
    my_dict[r"pack_tailhead_P190_0"+str(i)]["scan"+str(int(key[2]))]["P190_0"+str(i)+"_"+cell[0]]=tailhead #start of tail
    my_dict[r"pack_slope_P190_0"+str(i)]["scan"+str(int(key[2]))]["P190_0"+str(i)+"_"+cell[0]]=slope #slope of tail
    my_dict[r"pack_diameter_P190_0"+str(i)]["scan"+str(int(key[2]))]["P190_0"+str(i)+"_"+cell[0]]=diameter #diameter of semicircle obtained from half max
    my_dict[r"pack_shape_P190_0"+str(i)]["scan"+str(int(key[2]))]["P190_0"+str(i)+"_"+cell[0]]=shape #shape=representation of shape (0.75max-Rs)/(0.5max-Rs)
    my_dict[r"pack_shoulder_P190_0"+str(i)]["scan"+str(int(key[2]))]["P190_0"+str(i)+"_"+cell[0]]=shoulder #
    my_dict[r"pack_ymax_P190_0"+str(i)]["scan"+str(int(key[2]))]["P190_0"+str(i)+"_"+cell[0]]=ymax #maximum of Zimg

  #stack features of one pack together to save
  Feature=pd.concat([my_dict["pack_"+Feature_name+"_P190_0"+str(i)] for Feature_name in ["xofymax","intercept","diameter","tailhead","slope","shape","shoulder","ymax"]])
  currentDateTime = datetime.now().strftime("%m-%d-%Y %H-%M-%S %p")
  Feature.to_csv("Generated Features/P190_0"+str(i)+f"_features {currentDateTime}.csv", index = True)

In [ ]:
# @title
#extract features in pack P190-03
my_dict = {}
for i in [3]:
  #get filenames of EIS from all cells
  directory = r'Data/P190/P190_0'+str(i)+'/'
  all_files = glob.glob(os.path.join(directory,  r'EIS_*/2_EIS_*_P190_0'+str(i)+'_*'))
  n=len(all_files)

  #initialize dataframes for features for each pack
  for x in [r"pack_xofymax_P190_0"+str(i),r"pack_intercept_P190_0"+str(i),r"pack_diameter_P190_0"+str(i),r"pack_tailhead_P190_0"+str(i),\
            r"pack_slope_P190_0"+str(i),r"pack_shape_P190_0"+str(i),r"pack_shoulder_P190_0"+str(i),r"pack_ymax_P190_0"+str(i)]:
    my_dict[x] =pd.DataFrame(columns=["scan"+str(j) for j in [2,4,6,8]],index=["P190_0"+str(i)+"_"+str(j) for j in range(1,6)])


  for filename in all_files:
    data = pd.read_csv(filename, delimiter = ',', header = 0, encoding='latin1')

    pre=filename.split("/")
    key=pre[-1].split("_")
    cell=key[5].split(" ")


    x=data["Z' (Ohms)"].to_numpy()
    y=data['-Z" (Ohms)'].to_numpy()

    print(["scan"+key[2] + "_P190_" + key[4]+"_"+cell[0]])

    if [int(cell[0]),int(key[2])]==[4,2]:

      xofymax,intercept,tailhead,slope,diameter,shape,shoulder,ymax=Nyquist_feature_extract(x,y,500,0,Zreal_exclude=[0.05301404,0.05261355])
    elif [int(cell[0]),int(key[2])]==[4,8]:

      xofymax,intercept,tailhead,slope,diameter,shape,shoulder,ymax=Nyquist_feature_extract(x,y,500,0,defined_turn=0.0327)

    else:
      xofymax,intercept,tailhead,slope,diameter,shape,shoulder,ymax=Nyquist_feature_extract(x,y,500,0)


    my_dict[r"pack_xofymax_P190_0"+str(i)]["scan"+str(int(key[2]))]["P190_0"+str(i)+"_"+cell[0]]=xofymax #x of maximum of Zimg
    my_dict[r"pack_intercept_P190_0"+str(i)]["scan"+str(int(key[2]))]["P190_0"+str(i)+"_"+cell[0]]=intercept #intercept
    my_dict[r"pack_tailhead_P190_0"+str(i)]["scan"+str(int(key[2]))]["P190_0"+str(i)+"_"+cell[0]]=tailhead #start of tail
    my_dict[r"pack_slope_P190_0"+str(i)]["scan"+str(int(key[2]))]["P190_0"+str(i)+"_"+cell[0]]=slope #slope of tail
    my_dict[r"pack_diameter_P190_0"+str(i)]["scan"+str(int(key[2]))]["P190_0"+str(i)+"_"+cell[0]]=diameter #diameter of semicircle obtained from half max
    my_dict[r"pack_shape_P190_0"+str(i)]["scan"+str(int(key[2]))]["P190_0"+str(i)+"_"+cell[0]]=shape #shape=representation of shape (0.75max-Rs)/(0.5max-Rs)
    my_dict[r"pack_shoulder_P190_0"+str(i)]["scan"+str(int(key[2]))]["P190_0"+str(i)+"_"+cell[0]]=shoulder #
    my_dict[r"pack_ymax_P190_0"+str(i)]["scan"+str(int(key[2]))]["P190_0"+str(i)+"_"+cell[0]]=ymax #maximum of Zimg

  #stack features of one pack together to save
  Feature=pd.concat([my_dict["pack_"+Feature_name+"_P190_0"+str(i)] for Feature_name in ["xofymax","intercept","diameter","tailhead","slope","shape","shoulder","ymax"]])
  currentDateTime = datetime.now().strftime("%m-%d-%Y %H-%M-%S %p")
  Feature.to_csv("Generated Features/P190_0"+str(i)+f"_features {currentDateTime}.csv", index = True)

['scan08_P190_03_5']
['scan02_P190_03_1']
['scan02_P190_03_2']
['scan02_P190_03_3']
['scan02_P190_03_4']
['scan02_P190_03_5']
['scan04_P190_03_1']
['scan04_P190_03_2']
['scan04_P190_03_3']
['scan04_P190_03_4']
['scan04_P190_03_5']
['scan06_P190_03_1']
['scan06_P190_03_2']
['scan06_P190_03_3']
['scan06_P190_03_4']
['scan06_P190_03_5']
['scan08_P190_03_1']


<ipython-input-38-86a365e9ef65>:54: RankWarning:

Polyfit may be poorly conditioned

<ipython-input-38-86a365e9ef65>:54: RankWarning:

Polyfit may be poorly conditioned

<ipython-input-38-86a365e9ef65>:54: RankWarning:

Polyfit may be poorly conditioned

<ipython-input-38-86a365e9ef65>:54: RankWarning:

Polyfit may be poorly conditioned

<ipython-input-38-86a365e9ef65>:54: RankWarning:

Polyfit may be poorly conditioned

<ipython-input-38-86a365e9ef65>:54: RankWarning:

Polyfit may be poorly conditioned

<ipython-input-38-86a365e9ef65>:54: RankWarning:

Polyfit may be poorly conditioned

<ipython-input-38-86a365e9ef65>:54: RankWarning:

Polyfit may be poorly conditioned

<ipython-input-38-86a365e9ef65>:54: RankWarning:

Polyfit may be poorly conditioned

<ipython-input-38-86a365e9ef65>:54: RankWarning:

Polyfit may be poorly conditioned

<ipython-input-38-86a365e9ef65>:54: RankWarning:

Polyfit may be poorly conditioned

<ipython-input-38-86a365e9ef65>:54: RankWarning:

Polyfit may be 

['scan08_P190_03_2']
['scan08_P190_03_3']
['scan08_P190_03_4']


<ipython-input-38-86a365e9ef65>:54: RankWarning:

Polyfit may be poorly conditioned

<ipython-input-38-86a365e9ef65>:54: RankWarning:

Polyfit may be poorly conditioned

<ipython-input-38-86a365e9ef65>:54: RankWarning:

Polyfit may be poorly conditioned



In [ ]:
# @title
#extract features in pack P190-04
my_dict = {}
for i in [4]:
  #get filenames of EIS from all cells
  directory = r'Data/P190/P190_0'+str(i)+'/'
  all_files = glob.glob(os.path.join(directory,  r'EIS_*/2_EIS_*_P190_0'+str(i)+'_*'))
  n=len(all_files)

  #initialize dataframes for features for each pack
  #features: ymax=maximum of Zimg, intercept=intercept with x-axis,diameter=diameter of semicircle, tailhead=start of linear tail, slope=slope of linear tail, shape=representation of shape (center-0.5max)/(center-0.75max)
  for x in [r"pack_xofymax_P190_0"+str(i),r"pack_intercept_P190_0"+str(i),r"pack_diameter_P190_0"+str(i),r"pack_tailhead_P190_0"+str(i),r"pack_slope_P190_0"+str(i),r"pack_shape_P190_0"+str(i),r"pack_shoulder_P190_0"+str(i),r"pack_ymax_P190_0"+str(i)]:
    my_dict[x] =pd.DataFrame(columns=["scan"+str(j) for j in [2,4,6,8]],index=["P190_0"+str(i)+"_"+str(j) for j in range(1,6)])


  for filename in all_files:
    data = pd.read_csv(filename, delimiter = ',', header = 0, encoding='latin1')

    pre=filename.split("/")
    key=pre[-1].split("_")
    cell=key[5].split(" ")


    x=data["Z' (Ohms)"].to_numpy()
    y=data['-Z" (Ohms)'].to_numpy()

    print(["scan"+key[2] + "_P190_" + key[4]+"_"+cell[0]])



    if [int(cell[0]),int(key[2])]==[2,4]:

      xofymax,intercept,tailhead,slope,diameter,shape,shoulder,ymax=0,0,0,0,0,0,0,0
    elif [int(cell[0]),int(key[2])]==[3,8]:
      xofymax,intercept,tailhead,slope,diameter,shape,shoulder,ymax=Nyquist_feature_extract(x,y,500,0,defined_turn=0.03)
    elif [int(cell[0]),int(key[2])] in [[2,8],[4,8],[5,8]]:
      xofymax,intercept,tailhead,slope,diameter,shape,shoulder,ymax=Nyquist_feature_extract(x,y,500,0,defined_turn=0.034)
    elif [int(cell[0]),int(key[2])]==[4,2]:
      xofymax,intercept,tailhead,slope,diameter,shape,shoulder,ymax=Nyquist_feature_extract(x,y,500,0,defined_turn=0.037)
    elif [int(cell[0]),int(key[2])]==[5,2]:
      xofymax,intercept,tailhead,slope,diameter,shape,shoulder,ymax=Nyquist_feature_extract(x,y,500,0,defined_turn=0.038)
    else:
      xofymax,intercept,tailhead,slope,diameter,shape,shoulder,ymax=Nyquist_feature_extract(x,y,500,0)

    my_dict[r"pack_xofymax_P190_0"+str(i)]["scan"+str(int(key[2]))]["P190_0"+str(i)+"_"+cell[0]]=xofymax #x of maximum of Zimg
    my_dict[r"pack_intercept_P190_0"+str(i)]["scan"+str(int(key[2]))]["P190_0"+str(i)+"_"+cell[0]]=intercept #intercept
    my_dict[r"pack_tailhead_P190_0"+str(i)]["scan"+str(int(key[2]))]["P190_0"+str(i)+"_"+cell[0]]=tailhead #start of tail
    my_dict[r"pack_slope_P190_0"+str(i)]["scan"+str(int(key[2]))]["P190_0"+str(i)+"_"+cell[0]]=slope #slope of tail
    my_dict[r"pack_diameter_P190_0"+str(i)]["scan"+str(int(key[2]))]["P190_0"+str(i)+"_"+cell[0]]=diameter #diameter of semicircle obtained from half max
    my_dict[r"pack_shape_P190_0"+str(i)]["scan"+str(int(key[2]))]["P190_0"+str(i)+"_"+cell[0]]=shape #shape=representation of shape (0.75max-Rs)/(0.5max-Rs)
    my_dict[r"pack_shoulder_P190_0"+str(i)]["scan"+str(int(key[2]))]["P190_0"+str(i)+"_"+cell[0]]=shoulder #
    my_dict[r"pack_ymax_P190_0"+str(i)]["scan"+str(int(key[2]))]["P190_0"+str(i)+"_"+cell[0]]=ymax #maximum of Zimg

  #stack features of one pack together to save
  Feature=pd.concat([my_dict["pack_"+Feature_name+"_P190_0"+str(i)] for Feature_name in ["xofymax","intercept","diameter","tailhead","slope","shape","shoulder","ymax"]])
  currentDateTime = datetime.now().strftime("%m-%d-%Y %H-%M-%S %p")
  Feature.to_csv("Generated Features/P190_0"+str(i)+f"_features {currentDateTime}.csv", index = True)

['scan08_P190_04_5']


<ipython-input-38-86a365e9ef65>:54: RankWarning:

Polyfit may be poorly conditioned



['scan02_P190_04_1']


<ipython-input-38-86a365e9ef65>:54: RankWarning:

Polyfit may be poorly conditioned



['scan02_P190_04_2']


...

<ipython-input-38-86a365e9ef65>:54: RankWarning:

Polyfit may be poorly conditioned



['scan08_P190_04_3']


<ipython-input-38-86a365e9ef65>:54: RankWarning:

Polyfit may be poorly conditioned



['scan08_P190_04_4']


<ipython-input-38-86a365e9ef65>:54: RankWarning:

Polyfit may be poorly conditioned

